In [38]:
import string
import pickle

import pandas as pd


# Load data

In [6]:
ROME_df = pd.read_csv('../referentiels/referentiel_ROME/20150921_arboprincipale28427_ROME.csv', index_col=0, sep='|', dtype=str)
OGR_df = pd.read_csv('../referentiels/referentiel_OGR/20150921_arboprincipale28427_OGR.csv', sep='|', dtype=str).set_index('OGR')
NAF_df = pd.read_csv('../referentiels/referentiel_NAF/naf2008_liste_n5_nouveau_header.csv', sep='|', encoding="utf-8").set_index(['NAF'])

In [24]:
data = '''
*** D1106 : Vente en alimentation ***

Vendeur / Vendeuse en boulangerie-pâtisserie
20560 Vendeur / Vendeuse en boulangerie-pâtisserie (45782.0)

Vendeur / Vendeuse en chocolaterie-confiserie
20567 Vendeur / Vendeuse en chocolaterie (2981.0)
20568 Vendeur / Vendeuse en confiserie (811.0)

Vendeur / Vendeuse en épicerie
20576 Vendeur / Vendeuse en épicerie (1467.0)
20577 Vendeur / Vendeuse en épicerie fine (2868.0)
20570 Vendeur / Vendeuse en crèmerie (434.0)
38776 Vendeur / Vendeuse en produits biologiques (2067.0)
20622 Vendeur / Vendeuse en produits diététiques (1062.0)

Vendeur / Vendeuse en alimentation générale
20525 Vendeur / Vendeuse de fruits et légumes (3062.0)
16388 Marchand / Marchande de fruits et légumes (51.0)
20530 Vendeur / Vendeuse de primeurs (249.0)
20540 Vendeur / Vendeuse en alimentation générale (3395.0)
14863 Epicier / Epicière (353.0)
20558 Vendeur / Vendeuse en boucherie (1443.0)
20559 Vendeur / Vendeuse en boucherie-charcuterie (2069.0)
20627 Vendeur / Vendeuse en rôtisserie (236.0)
20564 Vendeur / Vendeuse en charcuterie (4729.0)
20612 Vendeur / Vendeuse en poissonnerie (3842.0)
20623 Vendeur / Vendeuse en produits frais (1928.0)
20645 Vendeur / Vendeuse rayon fromages (1194.0)
20646 Vendeur / Vendeuse rayon traiteur (926.0)

Vendeur / Vendeuse en vins et spiritueux
20635 Vendeur / Vendeuse en vins et spiritueux (1913.0)


*** A1201 : Bûcheronnage et élagage ***
   
Bûcheron / Bûcheronne
11606 Bûcheron / Bûcheronne (1738.0)
12755 Commis / Commise de coupe (59.0)
17483 Ouvrier / Ouvrière d'exploitation forestière (244.0)
11607 Bûcheron élagueur / Bûcheronne élagueuse (589.0)
   
Elagueur / Elagueuse
14608 Elagueur / Elagueuse (1279.0)
14610 Elagueur-botteur / Elagueuse-botteuse (4.0)
38435 Homme / Femme de pied (134.0)
14609 Elagueur / Elagueuse espaces verts (408.0)
14611 Elagueur-grimpeur / Elagueuse-grimpeuse (1054.0)
11106 Arboriste grimpeur / grimpeuse (88.0)
15429 Grimpeur-élagueur / Grimpeuse-élagueuse (143.0)
14607 Ehouppeur / Ehouppeuse (0.0)

'''

# Parse data

In [32]:
lines = data.split('\n')

def begin_ROME(line_number, line):
    words = line.split(' ')
    assert words[0] == '***', 'Erreur! J\'attendais "*** Z1234 xxx ***" (ligne {})'.format(line_number)
    current_ROME = words[1]
    assert current_ROME in ROME_df.index, 'Erreur! Code ROME "{}" inconnu (ligne {})'.format(current_ROME, line_number)
    return current_ROME

current_ROME = None
current_group_OGRs = None
current_group_name = None
groups = {}
for line_number, line in enumerate(lines):
    line = line.strip()
    
    if line == '':
        if current_group_name:
            assert current_group_OGRs, 'Erreur! Le groupe "{}" est vide (ligne {})'.format(current_group_name, line_number)
            groups[current_ROME].append({'label': current_group_name, 'OGRs': current_group_OGRs})
            current_group_name = None
            current_group_OGRs = None
    else:
        if current_group_name:
            words = line.split(' ')
            OGR = words[0]
            assert OGR in set(OGR_df[OGR_df.ROME == current_ROME].index), 'Erreur! Le code OGR "{}" ne fais pas partie du code ROME "{}" (ligne {})'.format(OGR, current_ROME, line_number)
            current_group_OGRs.append(OGR)
        elif current_ROME:
            words = line.split(' ')
            if words[0] == '***':
                list_OGRs = []
                for group in groups[current_ROME]:
                    list_OGRs += group['OGRs']
                set_OGRs_in_data = set(list_OGRs)
                assert len(set_OGRs_in_data) == len(list_OGRs), 'Erreur! Un code OGR est présent dans plusieurs groupes du code ROME "{}" (ligne {})'.format(current_ROME, line_number)
                set_OGRs = set(OGR_df[OGR_df.ROME == current_ROME].index)
                assert set_OGRs_in_data == set_OGRs, 'Erreur! Tous les codes OGR ne sont pas classés pour le code ROME "{}" : {} vs {} (ligne {})'.format(current_ROME, set_OGRs_in_data, set_OGRs, line_number)
            
                current_ROME = begin_ROME(line_number, line)
                assert current_ROME not in groups, 'Erreur! Le ROME "{}" est déjà défini (ligne {})'.format(current_ROME, line_number)
                groups[current_ROME] = []
            else:
                current_group_name = line
                current_group_OGRs = []

        else:
            current_ROME = begin_ROME(line_number, line)
            assert current_ROME not in groups, 'Erreur! Le ROME "{}" est déjà défini (ligne {})'.format(current_ROME, line_number)
            groups[current_ROME] = []

            
        
assert current_group_name not in {group['label'] for group in groups[current_ROME]}, 'Erreur! Le nom "{}" est déjà définiTous les codes OGR ne sont pas classés pour le code ROME "{}" (ligne {})'.format(current_ROME, line_number)


# Nommage des regroupements

In [30]:
suffixes = string.ascii_lowercase
suffixes


'abcdefghijklmnopqrstuvwxyz'

In [35]:

for ROME, ROME_groups in groups.items():
    for i, group in enumerate(ROME_groups):
        label = group['label']
        OGRs = group['OGRs']
        group['name'] = ROME + suffixes[i]

In [36]:
groups

{'A1201': [{'OGRs': ['11606', '12755', '17483', '11607'],
   'label': 'Bûcheron / Bûcheronne',
   'name': 'A1201a'},
  {'OGRs': ['14608',
    '14610',
    '38435',
    '14609',
    '14611',
    '11106',
    '15429',
    '14607'],
   'label': 'Elagueur / Elagueuse',
   'name': 'A1201b'}],
 'D1106': [{'OGRs': ['20560'],
   'label': 'Vendeur / Vendeuse en boulangerie-pâtisserie',
   'name': 'D1106a'},
  {'OGRs': ['20567', '20568'],
   'label': 'Vendeur / Vendeuse en chocolaterie-confiserie',
   'name': 'D1106b'},
  {'OGRs': ['20576', '20577', '20570', '38776', '20622'],
   'label': 'Vendeur / Vendeuse en épicerie',
   'name': 'D1106c'},
  {'OGRs': ['20525',
    '16388',
    '20530',
    '20540',
    '14863',
    '20558',
    '20559',
    '20627',
    '20564',
    '20612',
    '20623',
    '20645',
    '20646'],
   'label': 'Vendeur / Vendeuse en alimentation générale',
   'name': 'D1106d'},
  {'OGRs': ['20635'],
   'label': 'Vendeur / Vendeuse en vins et spiritueux',
   'name': 'D1106e'}]

In [39]:
with open('../decoupage_ROME.pickle', 'wb') as f:
    pickle.dump(groups, f)